In [1]:
import cv2 as cv
import numpy as np
import sklearn as skl
import scipy as cp
import matplotlib.pyplot as plt
import tensorflow as tn
import torch as pt
import os
import csv
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import manifold, neighbors, metrics
from scipy import signal



In [22]:

currentLocation = os.getcwd()
path = currentLocation + '/data/sign_mnist_train.csv'
train = pd.read_csv(path)
test = pd.read_csv(path)
#train.head()
labels = train['label'].values
unique_val = np.array(labels)
np.unique(unique_val)
label_binrizer = LabelBinarizer()
labels = label_binrizer.fit_transform(labels)
train.drop('label', axis = 1, inplace = True)
images = train.values


pathTest = currentLocation + '/data/sign_mnist_test.csv'
test = pd.read_csv(pathTest)
labelsTest = test['label'].values
unique_valTest = np.array(labelsTest)
np.unique(unique_valTest)
label_binrizerTest = LabelBinarizer()
labels = label_binrizer.fit_transform(labelsTest)
train.drop('label', axis = 1, inplace = True)
imagesTest = trainTest.values

KeyError: "['label'] not found in axis"

In [ ]:
#plt.style.use('rgb')
fig, axs = plt.subplots(1, 5, figsize=(15, 4), sharey=True)
for i in range(5): 
        axs[i].imshow(images[i].reshape(28,28))
fig.suptitle('Grayscale images')

In [19]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, stratify = labels, random_state = 7)



def augment(origData, origLabel, runs, strength):
    """
    Augments the data by naively aplying random gaussian noise to array, applies same strength to whole array
        :param original data, original labels, how many times to augment data, the strength of the noise
        :return: augmented list of data and labels
        """
    labelToAppend = origLabel
    dataToPermutate = origData
    for i in range(runs): #For each augmentation
        origLabel = np.vstack((origLabel,labelToAppend)) #Extend labels since wwe don't want to change them at all
        origData = np.vstack((origData, plusNoise(dataToPermutate, strength))) #Add data where noise have been added
        origLabel = np.vstack((origLabel,labelToAppend)) #Extend labels again
        origData = np.vstack((origData, negNoise(dataToPermutate, strength))) #Add data where noise have been subtracted
    return origData, origLabel

def plusNoise(inData, strength):
    """
    Add gausian white noise to data
        :param Data to apply noise to, strength of noise to apply
        :return: Data where noise have been added
        """
    data = inData
    for i in range(len(inData)): # For each element
        data[i] = inData[i]+np.random.normal(0, strength, size=(len(data[0]))) #Add noise
    return data

def negNoise(inData, strength):
    """
    Subtract gaussian white noise from data
        :param Data to subtract from, strength of noise
        :return: Data where noise have been subtracted
        """
    data = inData
    for i in range(len(inData)): # for each element
        data[i] = inData[i] + np.random.normal(0, strength, size=(len(data[0]))) # Remove noise
    return data
print(np.shape(y_train))
x_train, y_train = augment(x_train,y_train,4,0.01)
print(np.shape(y_train))

(19218, 24)
(172962, 24)


In [20]:
dt = tree.DecisionTreeClassifier()#Create model
dt.fit(x_train,y_train) #Train model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:

prediction = dt.predict(x_test) #Do prediction on test data
accuracy = metrics.accuracy_score(y_test, prediction) #Calculate accuracy from predictions
print("\n Accuracy: %f",accuracy)
print("\n Results from classifier:  \n %s \n" %( metrics.classification_report(y_test, prediction)))
print("\n Confussion matrix:\n %s" % metrics.confusion_matrix(y_test, prediction)) #PRint confusion matrix
#cnf_matrix = confusion_matrix(testLabels, prediction) #create matrix for plotting
#plot_confusion_matrix(cnf_matrix, classes=keys, title='Normalized confusion matrix')
#kfold = 10
#scores = cross_val_score(dt, images , labels, cv=10) #Do cross validation
#print("\n Cross validation score: \n")
#print(scores) #Report scores of cross validation
#print("\n Mean cv score: %f", np.mean(scores))


 Accuracy: %f 0.9061551535753308

 Results from classifier:  
               precision    recall  f1-score   support

           0       0.90      0.92      0.91       338
           1       0.91      0.89      0.90       303
           2       0.96      0.91      0.94       343
           3       0.92      0.88      0.90       359
           4       0.89      0.91      0.90       287
           5       0.89      0.93      0.91       361
           6       0.94      0.93      0.94       327
           7       0.90      0.90      0.90       304
           8       0.86      0.91      0.89       349
           9       0.91      0.87      0.89       334
          10       0.90      0.90      0.90       372
          11       0.89      0.91      0.90       317
          12       0.92      0.91      0.91       345
          13       0.95      0.92      0.94       359
          14       0.97      0.96      0.96       326
          15       0.96      0.92      0.94       384
          16     

ValueError: multilabel-indicator is not supported

In [23]:
prediction = dt.predict(test) #Do prediction on test data
accuracy = metrics.accuracy_score(labelsTest, prediction) #Calculate accuracy from predictions
print("\n Accuracy: %f",accuracy)
print("\n Results from classifier:  \n %s \n" %( metrics.classification_report(labelsTest, prediction)))


ValueError: Number of features of the model must match the input. Model n_features is 784 and input n_features is 785 